In [ ]:
from avidares.avidares import create_resource_video
import numpy as np
import seaborn as sb
import matplotlib as plt
import matplotlib.patches as patches
from IPython.display import HTML

In [ ]:
# Define a dictionary with the configuration settings
# There are two separate (well, three) separate configurations
# That we can set up here: one for the animation, one for the experiment,
# and one for the data transform.  The transform itself is simply some
# kind of function.  The others are dictionaries that get mapped into
# values as the experiment is run and the animation is processed


# These are just some 'globals' that will make configuration easier
inflow_box = np.array([[5,5],[15,15]])     # Upper right and lower left coordinates...
outflow_box = np.array([[30,30], [40,40]]) # for our respective in and outflow boxes
world = [60,60]  # We're going to use this quite a bit in the configuration below


# Configuration settings for the animation itself
anim_cfg = {
    'interval':50, # The speed of the animation in ms
    'cmap':sb.cubehelix_palette(start=2, rot=0, hue=1, dark=0.10, light=0.90, gamma=1, n_colors=16), #Colormap
}


# Configuration settings for the Avida experiment itself
expr_inflow_outflow = {
     'cwd':'default_config',  # Where can we find Avida and the default configuration files?
     'world':world, # REQUIRED; overrides the configuration files.  Data transforms rely upon this.
     'args':'-s 10',  # Command line arguments   
     
    'environment':  
     '\
         RESOURCE food:geometry={geometry}:initial=0:inflow=1\n\
         RESOURCE food:inflowX1={ifl_x1}:inflowX2={ifl_x2}:inflowY1={ifl_y1}:inflowY2={ifl_y2}\n\
         RESOURCE food:outflowX1={ofl_x1}:outflowX2={ofl_x2}:outflowY1={ofl_y1}:outflowY2={ofl_y2}:outflow=1.0\n\
     ',  # The environment file contents; if included in the config dictionary, use the default file
         # Note that we're templating the geometry parameter so we can change it using string format(...)
         # MAKE SURE TO END EACH LINE WITH \n FOR A NEWLINE AND \ TO TELL PYTHON TO CONTINUE PARSING THE NEXT LINE
    
    'events': 
    '\
        u begin Inject default-heads.org\n\
        u 0:100:end PrintSpatialResources resources.dat\n\
        u 50000 exit\n\
    ',  # The events file; if not included in the config directory, use the default file
}


def rectangle_xy_to_wh(box):
    '''
    Matplotlib's Rectangle patch uses the lower left coordinate alone
    with a width and height to construct itself.  This method simply
    converts a box in the form of [ [x1,y1], [x2,y2] ] to a compatible
    set of values
    '''
    left, right = box[0][0], box[1][0]
    top, bottom = box[0][1], box[1][1]
    width = right-left
    height = top-bottom
    return left,bottom,width,height


def highlight_flows(inflow_box, outflow_box):
    """
    Post-plotting generator to highlight the inflow and outflow boxes.
    Inflow is cyan and outflow is magenta.
    
    :param inflow_box:  A (x1,y1),(x2,y2) coordinate pair for the inflow box
    :param outflow_box: A (x1,y1),(x2,y2) coordinate pair for the outflow box
    
    :return: The same axes
    """
    # Convert our upper-left and lower-right coordinates into something compatible with
    # matplotlib's rectangle patch
    i_left,i_bottom,i_width,i_height = rectangle_xy_to_wh(inflow_box)
    o_left,o_bottom,o_width,o_height = rectangle_xy_to_wh(outflow_box)
    
    ax = None
    while True:
        ax = yield ax
        inflow = patches.Rectangle((i_left,i_bottom), i_width, i_height, edgecolor='cyan', fill=False)
        outflow = patches.Rectangle((o_left, o_bottom), o_width, o_height, edgecolor='magenta', fill=False)
        ax.add_patch(inflow)
        ax.add_patch(outflow)
    

# Run, plot, and display animation
for geom in ['torus', 'grid']:
    expr_cfg = expr_inflow_outflow.copy()  #Create a copy of our templated configuration
    
    # Change our title to match what we're plotting
    anim_cfg['title'] = 'Boxed Inflow/Outflow using {geometry}\nInflow=cyan, Outflow=magenta'.format(geometry=geom)
    
    # Fill in our "templated" environment configuration
    # with the proper geometry and in/outflow box coordintaes
    expr_cfg['environment'] = expr_cfg['environment'].format(
        geometry=geom, 
        ifl_x1=inflow_box[0][0], ifl_y1=inflow_box[0][1], ifl_x2=inflow_box[1][0], ifl_y2=inflow_box[1][1],
        ofl_x1=outflow_box[0][0], ofl_y1=outflow_box[0][1], ofl_x2=outflow_box[1][0], ofl_y2=outflow_box[1][1])  

    
    
    # We're going to use a generator to patch our drawings with the inflow and outflow
    # boxes.  In order to avoid having to pass the box-coordinates around, we're going
    # to use a generator that allows us to send a value (in this case the working axes)
    # that is initialized with our static box-coordinates.  To do this, we have to
    # first create an instance of the generator 'highlight_flows' and initialize it with
    # the arguments we'd like to store there.  Next we have to actually initiate the
    # generator by sending it a None value, so it is ready to work with us when our
    # animate frame function calls it.
    patch_flow_boxes = highlight_flows(inflow_box=inflow_box, outflow_box=outflow_box)
    patch_flow_boxes.send(None)
    
    # Add our post-plotting function to the animation configuration
    anim_cfg['post_plot_fn'] = patch_flow_boxes
    
    # Render; display(...) must be called since we're in a loop    
    display(HTML(create_resource_video(expr_cfg, anim_cfg)))

